In [32]:
# 영상 다운로드
from pytube import YouTube
import cv2
import os

os.makedirs('project', exist_ok=True)
# 다운로드할 유튜브 영상의 링크
url = "https://www.youtube.com/watch?v=kW_z-NMuZIU"

# YouTube 객체 생성
yt = YouTube(url)

# 가장 높은 품질의 비디오 스트림 선택
video = yt.streams.get_highest_resolution()

# 다운로드 시작
video.download(filename='project/한국vs호주.mp4')

'c:\\Classes\\05_numpy_opencv\\project/한국vs호주.mp4'

In [33]:
# 동영상 편집 (영상 길이 조정)
import cv2

# 동영상 파일 경로
video_path = "project\한국vs호주.mp4"

# VideoCapture 객체를 생성하여 동영상 파일을 엽니다
cap = cv2.VideoCapture(video_path)

# 시작 시간과 끝 시간 설정 (73초부터 232초까지)
start_time = 73  # 시작 시간(초)
end_time = 232    # 끝 시간(초)

# 시작 프레임과 종료 프레임 계산
fps = cap.get(cv2.CAP_PROP_FPS)
start_frame = int(start_time * fps)
end_frame = int(end_time * fps)

# 동영상 파일을 저장할 VideoWriter 객체를 생성합니다
output_video_path = "project/한국vs호주_edited.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(output_video_path, fourcc, fps, (int(cap.get(3)), int(cap.get(4))))

# 시작 프레임으로 이동합니다
cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

# 동영상을 읽어서 자릅니다
frame_count = start_frame
while frame_count < end_frame:
    succ, frame = cap.read()
    if not succ:
        break
    writer.write(frame)
    frame_count += 1

# 객체들을 해제합니다
cap.release()
writer.release()

print("동영상 편집이 완료되었습니다.")

동영상 편집이 완료되었습니다.


In [2]:
# 영상 200 프레임당 이미지 캡처
from pytube import YouTube
import cv2
import os

cap = cv2.VideoCapture('project\한국vs호주_edited.mp4')

# 동영상 FPS 값을 조회
FPS = cap.get(cv2.CAP_PROP_FPS)
delay = int(30/FPS)

cnt=0
i=0
while cap.isOpened():
    succ, frame = cap.read()
    if not succ:
        print('캡처 실패')
        break

    # 화면 출력 + 저장
    cv2.imshow('frame', frame)

    # 200 프레임당 캡처 저장
    if cnt%50==0:
        cv2.imwrite(f'project/soccer_{i}.jpg', frame)
        i += 1
    cnt += 1
    
    if cv2.waitKey(delay) == 27:    # esc 클릭 시 break
        break
    
cap.release()
cv2.destroyAllWindows()

캡처 실패


In [3]:
from glob import glob
import cv2
import numpy as np
import os

red_path = os.makedirs('project/Red', exist_ok=True)
yellow_path = os.makedirs('project/Yellow', exist_ok=True)
red_yellow_path = os.makedirs('project/Red_Yellow', exist_ok=True)

img_list = glob('project/*.jpg')
cnt=0
for img_path in img_list:
    fname = os.path.basename(img_path)      # 파일명 조회
    fname, ext = os.path.splitext(fname)    # 파일이름, 확장자를 분리해서 변수에 저장
    
    # 읽기
    img = cv2.imread(img_path)
    
    ### grayscale 변환
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ### 이진화
    _, img_bin = cv2.threshold(img_gray, 127, 255, cv2.THRESH_BINARY)

    ### 저장
    save_path1 = f'project/{fname}_binary_{cnt:03d}{ext}'   # 03d: 세자리 맞춤. 1 -> 001
    cv2.imwrite(save_path1, img_bin)

    # HSV
    ## 빨간 유니폼만 추출
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    low_red1 = np.array([0, 100, 100])
    high_red1 = np.array([15, 255, 255])
    low_red2 = np.array([165, 100, 100])
    high_red2 = np.array([180, 255, 255])
    mask1 = cv2.inRange(img_hsv, low_red1, high_red1)
    mask2 = cv2.inRange(img_hsv, low_red2, high_red2)
    mask = mask1 + mask2
    red_hsv = cv2.bitwise_and(img, img, mask=mask)
    save_path2 = f'project/Red/{fname}_hsv_R{cnt:03d}{ext}'
    cv2.imwrite(save_path2, red_hsv)

    # HSV
    ## 노란 유니폼만 추출
    low_yellow1 = np.array([20, 100, 100])
    high_yellow1 = np.array([35, 255, 255])
    mask3 = cv2.inRange(img_hsv, low_yellow1, high_yellow1)
    yellow_hsv = cv2.bitwise_and(img, img, mask=mask3)
    save_path2_2 = f'project/Yellow/{fname}_hsv_Y{cnt:03d}{ext}'
    cv2.imwrite(save_path2_2, yellow_hsv)

    # HSV
    ## RED + YELLOW
    mask = mask1 + mask2 + mask3
    ry_hsv = cv2.bitwise_and(img, img, mask=mask)
    save_path2_3 = f'project/Red_Yellow/{fname}_hsv_RY{cnt:03d}{ext}'
    cv2.imwrite(save_path2_3, ry_hsv)

    ## 밝기 변경 (30만큼 밝게)
    img_bright = cv2.add(img, 30)
    save_path3 = f'project/{fname}_bright_{cnt:03d}{ext}'
    cv2.imwrite(save_path3, img_bright)

    ## 엣지 검측
    img_edge = cv2.Canny(img_gray, 50, 150)
    save_path4 = f'project/{fname}_edge_{cnt:03d}{ext}'
    cv2.imwrite(save_path4, img_edge)

    cnt += 1